In [ ]:
import datetime as dt
import sys, os
import dateutil.relativedelta as rd
import json
import pathlib as pl
from typing import List
import warnings

import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import shapely.geometry as sg
import shapely.ops as so

DIR = pl.Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk


warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

DATA = DIR / 'data'

In [ ]:
feed = gk.read_feed(DATA / "cairns_gtfs.zip", dist_units='km')
feed.append_dist_to_stop_times()
# def my_apply(group):
#     g = group.copy()
#     g["shape_dist_traveled"] = 1
#     return g
    
# (
#     feed.stop_times
#     .drop("shape_dist_traveled", axis=1)
#     .groupby("trip_id", group_keys=True)
#     .apply(my_apply, include_groups=False)
#     .reset_index()
#     .drop("level_1", axis=1)  # What did this column appear?
# )

In [ ]:
dates = feed.get_first_week()[:3]
stop_id = feed.stops["stop_id"].iat[0]
route_id = feed.routes["route_id"].iat[0]
display(feed.build_stop_timetable(stop_id, dates))
display(feed.build_route_timetable(route_id, dates))


In [ ]:
#path = DATA / 'cairns_gtfs.zip'
path = pl.Path.home() / "Desktop" / "gtfs.zip"
feed = gk.read_feed(path, dist_units='km')
feed.describe()

In [ ]:
feed.get_shapes(as_gdf=True)

In [ ]:
gk.geometrize_shapes(feed.shapes.iloc[:1])

In [ ]:
ts = feed.compute_trip_stats()
display(ts.head().T)
ts.to_csv(DATA / "cairns_trip_stats.csv", index=False)

In [ ]:
# Maybe a useful function to add
def compute_stop_time_stats(feed):
    if "shape_dist_traveled" in feed.stop_times.columns:
        st = feed.stop_times.copy()
    else:
        st = feed.append_dist_to_stop_times().stop_times
    f = st.sort_values(["trip_id", "stop_sequence"])

    # Choose strategy for calculating duration to next stop
    if (
        f["arrival_time"].count() > 0.5 * f["arrival_time"].shape[0] 
        and f["departure_time"].count() > 0.5 * f["departure_time"].shape[0]
    ):
        # Use stop departure to next stop arrival as duration
        f["atime"] = f["arrival_time"].map(gk.timestr_to_seconds)
        f["dtime"] = f["departure_time"].map(gk.timestr_to_seconds)
        f["duration_to_next_stop"] = f.groupby("trip_id")["atime"].shift(-1) - f["dtime"] 
    elif f["arrival_time"].count() > 0.5 * f["arrival_time"].shape[0]:
        # Use stop arrival to next stop arrival as duration
        f["atime"] = f["arrival_time"].apply(gk.timestr_to_seconds)
        f["duration_to_next_stop"] = f.groupby("trip_id")["atime"].diff().shift(-1) 
    else:
        # Nothing satisfactory, so just use departure times and get many NAs
        f["dtime"] = f["departure_time"].map(gk.timestr_to_seconds)
        f["duration_to_next_stop"] = f.groupby("trip_id")["dtime"].diff().shift(-1) 

    # convert to km or mi
    if gk.is_metric(feed.dist_units):
        g = gk.get_convert_dist(feed.dist_units, "km")
    else:
        g = gk.get_convert_dist(feed.dist_units, "mi")

    f["distance_to_next_stop"] = f.groupby('trip_id')['shape_dist_traveled'].diff().shift(-1)  # feed.dist_units
    f["speed_to_next_stop"] = f["distance_to_next_stop"].map(g) / (f["duration_to_next_stop"] / 3600) # km/h or mi/h
    return f.drop(["atime", "dtime"], axis=1)

compute_stop_time_stats(feed).head(38)
    


In [ ]:
g = feed.geometrize_stops()
p = g["geometry"].iat[0]
type(p)
q = sg.MultiPoint([p, p])
list(q.geoms)

In [ ]:
feed.map_trips(trip_ids[:1], include_stops=True, include_arrows=True)

In [ ]:
dates = feed.get_first_week()[6:]
activity = feed.compute_trip_activity(dates)
display(activity)

trip_ids = activity.loc[
    lambda x: x.filter(dates).sum(axis=1) > 0,
    "trip_id",
]
trip_ids

In [ ]:
feed.trips

In [ ]:
dates = feed.get_first_week()[:1] + ["20010101"]

path = DATA_DIR / "cairns_screen_lines.geojson"
screen_lines = gpd.read_file(path)
f = feed.compute_screen_line_counts(screen_lines, dates)
f

In [ ]:
path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)



In [ ]:
feed = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed.describe()

In [ ]:
shapes = feed.geometrize_shapes(use_utm=True)
shapes.crs
screen_lines = screen_lines.to_crs(shapes.crs)
screen_lines

In [ ]:
url = "http://transitfeeds.com/p/10-15-transit/936/latest/download"
gk.read_feed(url, dist_units="km").describe()

In [ ]:
(
    g1.crossing_time.equals(g2.crossing_time),
    g1.crossing_direction.equals(g2.orientation)
)

In [ ]:
feed.map_routes(feed.routes.route_id.iloc[:4], include_stops=False)
